<a href="https://colab.research.google.com/github/JocznHM/Inteligencia-artificial-CJHM/blob/main/Modelo_Temp_FtoC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Entrenamiento de modelo para Ejecución en una pagina web

Para poder ejecutar un modelo Keras dentro de una pagina web es necesario trabajar con Tensorflow.js y esta libreria es compatible con versiones de python menores a 3.10, siendo python 3.8 la versión más optima para trabajar.

Colab trabaja directamente con la siguiente versión:

In [3]:
!python --version

Python 3.10.12


Aunque Colab trabaje con python 3.10, un modelo entrenado con esta versión no es capaz de funcionar con la libreria tensorflow.js, la solución a esto es optar por utilizar un entorno virtual en el cual se puede tener una versión de python estable para tensorflow.js, en este caso python 3.8.xxx.

a continuación, la instalación del entorno virtual y dependencias para entrenar el modelo:

In [5]:
# 1. Instalar Python 3.8
!sudo apt-get update -y
!sudo apt-get install python3.8 python3.8-dev python3.8-venv -y

# 2. Crear un entorno virtual con Python 3.8
!python3.8 -m venv tfjs-venv

# 3. Activar el entorno virtual
!source tfjs-venv/bin/activate

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,032 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,600 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRel

Una vez creado el venv, para poder trabajar dentro de el, se debe usar rutas relativas de la siguiente manera:

!ruta/venv/comandos_python                                               
 ya en práctica queda de la siguiente manera:

In [6]:
# 4. Actualizar pip dentro del entorno virtual
!tfjs-venv/bin/pip install --upgrade pip

# 5. Instalar TensorFlow y TensorFlow.js en el entorno virtual
!tfjs-venv/bin/pip install tensorflow==2.8 tensorflowjs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of tensorflowjs to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of tensorflowjs to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
  Using cached whe

con el siguiente comando se verifica que dentro del venv se esta ejecutando una versión de python 3.8.xxxx y tambien que librerias estan instaladas dentro del venv:

In [8]:
!tfjs-venv/bin/python --version
!tfjs-venv/bin/pip list

Python 3.8.20
Package                      Version
---------------------------- -------------------
absl-py                      2.1.0
astunparse                   1.6.3
cached-property              1.5.2
cachetools                   5.5.0
certifi                      2024.8.30
charset-normalizer           3.4.0
chex                         0.1.7
dm-tree                      0.1.8
etils                        1.3.0
flatbuffers                  24.3.25
flax                         0.7.2
gast                         0.6.0
google-auth                  2.35.0
google-auth-oauthlib         0.4.6
google-pasta                 0.2.0
grpcio                       1.67.0
h5py                         3.11.0
idna                         3.10
importlib_metadata           8.5.0
importlib_resources          6.4.5
jax                          0.4.13
jaxlib                       0.4.13
keras                        2.8.0
Keras-Preprocessing          1.1.2
libclang                     18.1.1
Markdown      

En la siguiente parte se crea el modelo, aquí se tienen que hacer unos cambios. Dado que se tiene que trabajar dentro de un venv para poder entrenar el modelo con python 3.8.xxxx, se tiene que crear un archivo con la extensión py y despues executarlo dentro del venv para generar el modelo. Por otro lado, la librería tensorflowjs en un entorno de programación python nos permite guardar el modelo ya entrenado desde el código sin necesidad de hacer uso de la termial de comandos, la pieza de codigo que se agrega al codigo fuente del modelo crea el archivo json y los bin necesarios para que el modelo pueda ser importado desde una pagina web con javascript.                                     
El código quedaria de la siguiente manera:

In [9]:
%%writefile entrenamiento_modelo.py
import tensorflow as tf
import tensorflowjs as tfjs
import numpy as np

# Datos de entrenamiento
celcius = np.array([-40, -10, 0, 8, 15, 22, 38], dtype=float)
fahrenheit = np.array([-40, 14, 32, 46, 59, 72, 100], dtype=float)

input_layer = tf.keras.layers.Input(shape=(1,), name='input')  # Capa de entrada
capa1 = tf.keras.layers.Dense(units=3, activation='relu', name='dense_1')(input_layer)  # Capa oculta 1
capa2 = tf.keras.layers.Dense(units=3, activation='relu', name='dense_2')(capa1)  # Capa oculta 2
output_layer = tf.keras.layers.Dense(units=1, name='output')(capa2)  # Capa de salida

# Creación del modelo
modelo = tf.keras.Model(inputs=input_layer, outputs=output_layer, name='modelo_temperatura')

# Compilación del modelo
modelo.compile(optimizer=tf.keras.optimizers.Adam(0.1), loss='mean_squared_error')

# Entrenamiento del modelo
print("Entrenando el modelo...")
modelo.fit(fahrenheit, celcius, epochs=1000, verbose=0)
print("Modelo entrenado!")

# Predicción con un nuevo valor
resultado = modelo.predict(np.array([100.0]))
print(f"Predicción para 100°F: {resultado[0][0]}°C")

# Exportar el modelo para TensorFlow.js, como primer parametro se pasa el modelo
# Para despues indicarle la ruta donde se va guardar, en este caso la ruta relativa ./temperatura_tfjs_model
# Si la carpeta no existe el mismo comando va crearla
tfjs.converters.save_keras_model(modelo, './temperatura_tfjs_model')


Writing entrenamiento_modelo.py


Con el comando **%%writefile** entrenamiento_modelo.py dentro de un entorno python como jupyter notebook o google colab, podemos decirle que cree un archivo con todo el contendio de la celda, en este caso todo el bloque de código anterior.                                       
Ahora, solo queda ejecutar este código dentro del venv y se generaran los archivos del modelo entrenado:

In [10]:
!tfjs-venv/bin/python entrenamiento_modelo.py

2024-10-18 22:09:11.646731: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-10-18 22:09:11.646767: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-10-18 22:09:14.413633: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-10-18 22:09:14.413679: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2024-10-18 22:09:14.413714: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this 

Al final solo basta con descargar los archivos generados dentro de la carpeta que indicamos en el comando de guardado dentro del codigo y agregarlos dentro de la misma ruta del archivo html donde se vaya usar el modelo, el cdn correcto según el sitio oficial de tensorflow es el siguiente:                                                  

    <script src="https://cdn.jsdelivr.net/npm/@tensorflow/tfjs@latest/dist/tf.min.js"></script>

A continuación mi codigo del archivo html:                               
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Celcius a Fahrenheit con IA</title>
    <script src="https://cdn.jsdelivr.net/npm/@tensorflow/tfjs@latest/dist/tf.min.js"></script>
    <style>
        #resultado {
            font-weight: bold;
            font-size: 1.2rem;
            text-align: center;
        }
    </style>
</head>
<body>
    <h1>Convertidor de Celcius a Fahrenheit</h1>
    <label for="Fahrenheit">Fahrenheit: </label>
    <input type="number" id="Fahrenheit"/>
    <button onclick="cambiarCelcius()">Calcular</button>
    <p id="lbl-Fahrenheit"></p>
    <p id="resultado"></p>

    <script type="text/javascript">
        var modelo = null;
        (async () => {
            console.log("Cargando modelo...");
            try {
                modelo = await tf.loadLayersModel('./model.json');
                console.log("Modelo cargado...");
            } catch (error) {
                console.error("Error al cargar el modelo: ", error);
            }
        })();

        async function cambiarCelcius() {
            var Fahrenheit = document.getElementById("Fahrenheit").value;
            console.log("grados fahrenheit:" + Fahrenheit)
            document.getElementById("lbl-Fahrenheit").innerHTML = Fahrenheit;
            if (modelo != null) {
                var tensor = tf.tensor([parseFloat(Fahrenheit)], [1, 1]); // Crear tensor
                const prediccion = modelo.predict(tensor); // Realizar predicción
                const resultado = await prediccion.data(); // Obtener los datos del tensor
                console.log(resultado)
                document.getElementById("resultado").innerHTML = `${Fahrenheit}°F son ${resultado[0]}°C.`;
            } else {
                document.getElementById("resultado").innerHTML = "No se ha cargado el modelo, intenta de nuevo en un momento...";
            }
        }
    </script>

</body>
</html>